In [4]:
# spam_ham_bow_kr.py
# -*- coding: utf-8 -*-
"""
단어 빈도(BoW) 기반 한국어 스팸메일 분류기 (개선 전/후 비교 버전)
- CSV 열: '분류' (값: '스팸메일' 또는 '정상메일'), '메시지'
- 모델1(베이스라인): CountVectorizer + MultinomialNB
- 모델2(개선): 사용자 키워드 보강 + min_df 완화 + 같은 분류기
"""

import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.base import BaseEstimator, TransformerMixin
import joblib

# ---------- 0) 공통 설정 ----------
CSV_PATH = 'email_first150_KO_only.csv'   # <-- 네 CSV 경로
RANDOM_STATE = 42
TOKEN_PAT = r'(?u)[가-힣A-Za-z0-9<]+[가-힣A-Za-z0-9<>]*'

# 개선용 사용자 키워드(스팸/햄 모두 포함)
CUSTOM_KEYWORDS = [
    # 스팸 경향
    "무료","수신거부","클릭","투자","수익","광고","경품","쿠폰","당첨","지급","링크",
    # 정상(업무/일상) 경향
    "회의","보고","첨부","자료","안건","검토","프로젝트","일정","확인","미팅","송부"
]

# ---------- 1) 간단 전처리 ----------
class SimpleCleaner(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.url = re.compile(r'https?://\S+|www\.\S+')
        self.email = re.compile(r'[\w\.-]+@[\w\.-]+\.\w+')
        self.phone = re.compile(r'\b\d{2,4}[- ]?\d{3,4}[- ]?\d{4}\b')
        self.num = re.compile(r'\d+')
        self.multispace = re.compile(r'\s+')

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        cleaned = []
        for s in X:
            s = str(s)
            s = self.url.sub(' <URL> ', s)
            s = self.email.sub(' <EMAIL> ', s)
            s = self.phone.sub(' <PHONE> ', s)
            s = self.num.sub(' <NUM> ', s)
            s = re.sub(r'[^가-힣A-Za-z0-9<>\s]', ' ', s)
            s = self.multispace.sub(' ', s).strip()
            cleaned.append(s)
        return cleaned

# ---------- 2) 데이터 로드 ----------
df = pd.read_csv(CSV_PATH, encoding='utf-8-sig')  # cp949면 encoding='cp949'
df = df.rename(columns={'분류': 'label', '메시지': 'text'})
df = df[['label', 'text']].dropna()

label_map = {'스팸메일': 1, '정상메일': 0}
df['y'] = df['label'].map(label_map)

X_train, X_test, y_train, y_test = train_test_split(
    df['text'], df['y'], test_size=0.2, random_state=RANDOM_STATE, stratify=df['y']
)

# ---------- 3) 베이스라인 파이프라인 ----------
baseline = Pipeline([
    ('clean', SimpleCleaner()),
    ('vect', CountVectorizer(
        token_pattern=TOKEN_PAT,
        ngram_range=(1, 2),
        min_df=2,
        max_df=0.95
    )),
    ('clf', MultinomialNB(alpha=0.5))
])

# ---------- 4) 개선형 파이프라인 ----------
# 아이디어: 사용자 키워드를 코퍼스에 주입하여 사전에 반드시 포함되게 하고,
# min_df를 1로 내려 희귀하지만 중요한 업무/스팸 단어를 보존
improved_cleaner = SimpleCleaner()
X_train_clean = improved_cleaner.fit_transform(X_train)

# 벡터라이저를 따로 fit하여 사용자 키워드가 포함되도록 "추가 문서"로 합쳐 학습
improved_vect = CountVectorizer(
    token_pattern=TOKEN_PAT,
    ngram_range=(1, 2),
    min_df=1,          # 완화
    max_df=0.95
)
# 사용자 키워드 문서를 인위적으로 추가 (공백으로 join해서 하나의 문장 취급)
extended_corpus = list(X_train_clean) + [' '.join(CUSTOM_KEYWORDS)]
improved_vect.fit(extended_corpus)

# 개선형 분류기 학습(파이프라인처럼 동작)
improved_clf = MultinomialNB(alpha=0.5)
improved_clf.fit(improved_vect.transform(X_train_clean), y_train)

# ---------- 5) 평가 함수 ----------
def evaluate(title, y_true, y_pred):
    print(f'\n[{title}] 정확도:', round(accuracy_score(y_true, y_pred), 4))
    print(f'\n[{title}] 분류 리포트\n', classification_report(
        y_true, y_pred, target_names=['정상메일', '스팸메일']))
    print(f'\n[{title}] 혼동 행렬\n', confusion_matrix(y_true, y_pred))

def print_top_spam_terms(title, vectorizer, classifier, top_k=20):
    feature_names = vectorizer.get_feature_names_out()
    log_prob = classifier.feature_log_prob_
    spam_rank = np.argsort((log_prob[1] - log_prob[0]))[::-1]
    print(f'\n[{title}] 스팸에 강하게 기여한 상위 {top_k} 단어/바이그램')
    for idx in spam_rank[:top_k]:
        print(feature_names[idx])

# ---------- 6) 베이스라인 평가 ----------
baseline.fit(X_train, y_train)
pred_base = baseline.predict(X_test)
evaluate('베이스라인', y_test, pred_base)

vect_base = baseline.named_steps['vect']
clf_base = baseline.named_steps['clf']
print_top_spam_terms('베이스라인', vect_base, clf_base, top_k=20)

# ---------- 7) 개선형 평가 ----------
X_test_clean = improved_cleaner.transform(X_test)
pred_impr = improved_clf.predict(improved_vect.transform(X_test_clean))
evaluate('개선형(사용자 키워드+min_df=1)', y_test, pred_impr)

print_top_spam_terms('개선형', improved_vect, improved_clf, top_k=20)

# ---------- 8) 예측 데모(10개) ----------
samples = [
    "무료 쿠폰 지금 받기! 링크 클릭하세요",           # 스팸
    "내일 12시에 미팅 가능한가요?",                 # 정상
    "080-XXXX 수신거부/광고 안내",                 # 스팸
    "전송 표준요금 수신 1.50",                      # 스팸
    "프로젝트 회의 자료 첨부합니다.",               # 정상
    "지금 투자하면 200% 수익 보장!",               # 스팸
    "오늘 저녁 약속 시간 다시 알려주세요.",         # 정상
    "무료 영화 티켓 받으세요 ▶ 클릭",               # 스팸
    "택배가 도착했습니다. 확인 부탁드립니다.",        # 정상
    "긴급: 계정 보안을 위해 비밀번호 재설정 필요",    # 스팸
]

print('\n[샘플 예측 - 베이스라인]')
for s in samples:
    p = baseline.predict_proba([s])[0, 1]
    print(f'{s}  --> 스팸확률={p:.3f}')

print('\n[샘플 예측 - 개선형]')
for s in samples:
    p = improved_clf.predict_proba(improved_vect.transform(improved_cleaner.transform([s])))[0, 1]
    print(f'{s}  --> 스팸확률={p:.3f}')

# ---------- 9) 모델 저장 ----------
joblib.dump(baseline, 'spam_ham_bow_baseline.joblib')
joblib.dump({'cleaner': improved_cleaner, 'vectorizer': improved_vect, 'clf': improved_clf},
            'spam_ham_bow_improved.joblib')
print("\n모델 저장 완료:",
      "spam_ham_bow_baseline.joblib / spam_ham_bow_improved.joblib")



[베이스라인] 정확도: 0.9

[베이스라인] 분류 리포트
               precision    recall  f1-score   support

        정상메일       0.92      0.96      0.94        25
        스팸메일       0.75      0.60      0.67         5

    accuracy                           0.90        30
   macro avg       0.84      0.78      0.80        30
weighted avg       0.89      0.90      0.90        30


[베이스라인] 혼동 행렬
 [[24  1]
 [ 2  3]]

[베이스라인] 스팸에 강하게 기여한 상위 20 단어/바이그램
로
<num> 로
무료
p
<num> p
당첨
전화 코드
코드
회신
으로
세
분당 <num>
로 전화
분당
p <num>
긴급
<num> 세
<num> 으로
전송
세 이상

[개선형(사용자 키워드+min_df=1)] 정확도: 0.9333

[개선형(사용자 키워드+min_df=1)] 분류 리포트
               precision    recall  f1-score   support

        정상메일       0.96      0.96      0.96        25
        스팸메일       0.80      0.80      0.80         5

    accuracy                           0.93        30
   macro avg       0.88      0.88      0.88        30
weighted avg       0.93      0.93      0.93        30


[개선형(사용자 키워드+min_df=1)] 혼동 행렬
 [[24  1]
 [ 1  4]]

[개선형] 스팸에 강하게 기여한 상위 20